# Vector elements & Betti numbers

We have already seen the Lagrange and the DG finite element spaces. They are both spaces of scalar-valued functions. How can we approximate solutions of boundary value problems that are vector fields?

We could certainly use multiple copies of the scalar finite element spaces (i.e., a product of  scalar finite element spaces) to approximate a vector field component by component. However, in some cases, distinctly better methods can be constructed using other vector finite element spaces, such as the Raviart-Thomas space or the Nedelec space, both spaces of vector fields that we shall see in this notebook. We will expand on their utility in certain  boundary value problems in later lectures.

Actually, there are more than just utilitarian reasons for studying these vector finite element spaces. They are connected through rich and beautiful discrete structures that other common finite-dimensional spaces lack. As a prelude to such things, we will see in this notebook how a topological invariant emerges when considering curl as an operator from the Nedelec space in two space dimensions.

## The Raviart-Thomas space

$\newcommand{\om}{\varOmega}
\newcommand{\oh}{\varOmega_h}
\newcommand{\dive}{\mathop{\mathrm{div}}}
\newcommand{\R}{\mathbb{R}}
$
For a smooth vector field  $q: \om \to \R^2$, whose Cartesian ($x$ and $y$) components are $q_0, q_1$, respectively, writing $\partial_0, \partial_1$ for the partial derivatives in $x$ and $y$ direction, the divergence $\dive q = \partial_0 q_0 + \partial_1 q_1$ obviously makes sense as a smooth scalar function.  However, when $q$ is discontinuous across element interfaces, its divergence generally only makes sense as a Schwartz distribution.  Let $\oh$ denote a geometrically conforming triangulation of $\om$,  let the DG space on it be denoted by $P_p(\oh) = \prod_{K \in \oh} P_p(K),$ where $P_p(K)$ denote the set of polynomials of degree at most $p$, 
and let the jump of the normal component of a vector field $u$ across an interior facet 
$F = \partial K^+ \cap \partial K^-$ shared by elements $K^\pm \in \oh$ whose outward unit normals are $n^\pm$ be denoted by
&LeftDoubleBracket; $q\cdot n$ &RightDoubleBracket; $= q|_{K^+} \cdot n^+ + q|_{K^-} \cdot n^-$. In earlier lectures, we computed the distributional divergence of $q \in P_p(\oh)^2$,  from which the following result was immediate: 


**Theorem.** <font color=blue>A vector field $q$ with components in the DG space is in $H(\dive, \om)$ if the jump of its normal component across element interfaces vanish, i.e., 
a vector field $q \in P_p(\oh)^2$ is in $H(\dive, \om)$  if 
&nbsp; &LeftDoubleBracket; $q\cdot n$ &RightDoubleBracket; = 0.</font>

The Raviart-Thomas (RT) finite element space in two space dimensions is defined by 
$$
R_{h, p} =
\left\{
q\in H(\dive, \om) : \quad \forall K \in \oh, \quad  q|_K \in P_p(K)^2 + \begin{pmatrix} x \\ y \end{pmatrix} P_p(K) \; \right\}.
$$
By the theorem above, this means that $q \in R_{h, p}$ satisfies 
&LeftDoubleBracket; $q\cdot n$ &RightDoubleBracket; $=0$.  Therefore, anticipating functions  you will plot below, we note that any plot of a function in the RT space should show a continuously varying *normal component* across the interior  mesh facets.

The continuity of the normal component is a useful property to have when we know that the  function being approximated has that same normal continuity property. Often, when $q: \om \to \R^2$ is a vector field representing a physical *flux*, its normal component remains smooth across a material interface even when its other components jump across the interface.

#### Interpolation

For example, consider the vector field 
$$
q = 
\left\{
\begin{aligned}
& y\,\sin(x)  && x \le 1/2
\\
& (1-y)\,\sin(x) && x > 1/2.
\end{aligned}
\right.
$$
Across the interface $x=1/2$, its normal component (the $x$-component in this case) is continuous, while its tangential component is discontinuous. Let's plot $q$.

In [ ]:
import ngsolve as ng
from ngsolve.webgui import Draw
from netgen.geom2d import CSG2d, Rectangle
from ngsolve import x, y, sin, GridFunction

geo = CSG2d()
Rlft = Rectangle(pmin=(0, 0),   pmax=(1/2, 1), mat='lft')
Rrgt = Rectangle(pmin=(1/2, 0), pmax=(1, 1),   mat='rgt')
geo.Add(Rlft)
geo.Add(Rrgt)
mesh = ng.Mesh(geo.GenerateMesh(maxh=1/4))

q = mesh.MaterialCF({'lft': (sin(x), y),
                     'rgt': (sin(x), 1-y)})

The normal component (which is the $x$ component in this case) of $q$ is smooth:

In [ ]:
Draw(q[0], mesh);

The tangential component of $q$ is discontinuous across the middle interface.

In [ ]:
Draw(q[1], mesh);

We can also plot $q$ as a vector field.

In [ ]:
Draw(q, mesh, vectors={'grid_size': 15});

If we try to interpolate such a function using a product of Lagrange spaces, then the discontinuity at the interface will generate large interpolation errors. Instead, knowing that the normal component is continuous, we can try to interpolate the function into the RT space.  This will liberate the tangential component of the approximation from continuity constraints, while preserving the normal continuity.  The implementation of the RT space $R_{h, p}$ in NGSolve is accessed using `ng.HDiv`. The `Set` method, as in other finite element spaces in NGSolve, can be used to perform the interpolation. 

In [ ]:
p = 5
R = ng.HDiv(mesh, order=p, RT=True)

In [ ]:
qh = GridFunction(R)
qh.Set(q)
print('Error in RT interpolation =', 
      ng.sqrt(ng.Integrate((q - qh)**2, mesh)))

We can compare this to what happens when we interpolate the same function in the product of two Lagrange finite element spaces. This space is readily accessible using the `ng.VectorH1` facility in NGSolve. (Alternately, you can make the Cartesian product of any two finite element spaces `V` and `W` in NGSolve using `ng.FESpace([V, W])`.) 

In [ ]:
V2 = ng.VectorH1(mesh, order=p)
qq = GridFunction(V2)
qq.Set(q)
print('Error in product Lagrange space interpolation =', 
      ng.sqrt(ng.Integrate((q - qq)**2, mesh)))

Clearly the error in the product space approach is several orders higher than that of the error produced by the RT space. The source of the large error while interpolating using  `ng.VectorH1` is immediately evident if you plot the $y$-component of the interpolant `qq` or the error.  The interpolant, being continuous in both components, has a difficult time approximating the discontinuous component.

In [ ]:
Draw(qq - q, mesh, 'Lagrange interpolation error', vectors={'grid_size': 25});

#### Shape functions

It is illustrative to visualize the basis functions of the RT space. We had gotten used to speaking of "hat functions" but that term is not useful for general finite elements. Basis functions with some locality properties can be constructed generally for abstract finite element spaces (more on that later) and they are called *shape functions*. Here is a visualization of a shape function of the *lowest order RT space* (the $p=0$ case).

In [ ]:
R = ng.HDiv(mesh, order=0, RT=True)

shapenumber = 15
shape = ng.GridFunction(R, name='shape')
shape.vec[:] = 0
shape.vec[shapenumber] = 1
Draw(shape, mesh, vectors={'grid_size': 30});

You can see any shape function you wish by revising the `shapenumber` in the code above. In all cases, you see that
- the shape functions are vector fields whose normal components are continuous, 
- they are supported on a patch of one or two elements, 
- their normal component is zero on all mesh facets except one.


Thus in the lowest order case, each shape function appears to be "associated" to a single mesh facet (an edge,  in our 2D mesh). If this edge is on the domain boundary, the  shape function is supported on one triangle. If the edge is an interior edge, the  shape function is supported on the two triangles which share the interior edge.

One can also directly see the connection between the number of mesh facets and the dimension of the lowest order RT space by querying both numbers:

In [ ]:
mesh.nfacet

In [ ]:
R.ndof

Of course, for higher $p$, a number of additional shape functions must be added. You can easily visualize the higher order shape functions by tweaking the above few lines of code.

## The 2D Nedelec finite element space

Consider the curl operator in two space dimensions, acting on a vector field $u: \om \to \R^2$, 
$$
\newcommand{\curl}{\mathop{\mathrm{curl}}}
\curl u = \partial_0 u_1  - \partial_1 u_0.
$$
You have likely worked with $\curl$ as an operator acting on three-dimensional (3D) vector fields. The above two-dimensional (2D) version is the obtained as the $z$-component of the 3D curl when applied to a vector field that has only $x$ and $y$ components. It takes a 2D vector field $u$ and produces a scalar field $\curl u$.

The 2D curl is intimately related to the 2D divergence. To see this, let $J_{\pi/2}$ denote the operator that rotates a vector clockwise by 90 degrees, i.e., 
$$
J_{\pi/2} 
\begin{pmatrix}
a \\ b
\end{pmatrix} = 
\begin{pmatrix}
b \\ -a
\end{pmatrix}.
$$
Now, if two vector fields $u$ and $q$ are related by  
$$
u = J_{\pi/2} q 
$$
then 
$$
q = 
\begin{pmatrix}
q_0 \\ q_1 
\end{pmatrix}, 
\qquad 
u = 
\begin{pmatrix}
u_0 \\ u_1
\end{pmatrix}= 
\begin{pmatrix}
q_1 \\ -q_0
\end{pmatrix}, 
$$
so  
$$
\dive q =  \curl u.
$$

Moreover, if $t$ denotes the counterclockwise unit tangent vector on element boundaries, then it is related to the unit outward normal $n$ at the same boundary point by 
$$
J_{\pi/2} n = t.
$$
Hence 
$$
u\cdot t = q \cdot n.
$$
Thus the condition that   &LeftDoubleBracket; $q\cdot n$ &RightDoubleBracket; = 0
is equivalent to the condition that the jump of the tangential component of $u$ vanish. 
Let $t$ denote counterclockwise unit tangent on boundaries of generic two-dimensional domains. The *jump of the tangential component* of a vector field $u$ across an interior facet  $F = \partial K^+ \cap \partial K^-$ shared by elements $K^\pm \in \oh$ whose counterclockwise unit tangents are $t^\pm$ is denoted by &LeftDoubleBracket; $u\cdot t$ &RightDoubleBracket; $= u|_{K^+} \cdot t^+ + u|_{K^-} \cdot t^-$. Letting $H(\curl, \om) = \{ u: \om \to \R^2 \big| \quad u_i \in L^2(\om), \curl u \in L^2(\om)\}$ and noting that $H(\curl, \om) = J_{\pi/2} H(\dive, \om)$, the prior theorem immediately implies the following.

**Corollary.** <font color=blue>A vector field $u$ with components in the DG space is in $H(\curl, \om)$ if the jump of its tangential component across element interfaces,  &LeftDoubleBracket; $u\cdot t$ &RightDoubleBracket;, vanish.
    

(Note that the above-mentioned rotational isomorphism between $H(\dive, \om)$ and $H(\curl, \om)$ does not hold in 3D. Indeed, there are more than one tangential components across mesh facets in 3D. The Nedelec space in 3D is a truly different space, not isomorphic to the 3D RT space. Postponing the 3D case for later discussions, let us proceed with the 2D case.)

We define the  Nedelec space in two dimensions as the RT space rotated:
$$
N_{h, p} := J_{\pi/2} R_{h,p}.  
$$
By our discussion above, $N_{h, p} \subset H(\curl, \om)$. It consists of piecewise polynomial vector fields that are tangentially continuous. Applying the rotation operator to the polynomial functions in our previous definition of $R_{h, p}$, we also find that $N_{h, p}$ can be equivalently described  by 
$$
N_{h, p} = 
\left\{
u\in H(\curl, \om) : \quad \forall K \in \oh, \quad 
u|_K \in P_p(K)^2 + \begin{pmatrix} -y \\ x \end{pmatrix} P_p(K) \; \right\}.
$$

NGSolve provides an implementation of the Nedelec space accessible through  `ng.HCurl` as follows.

In [ ]:
N = ng.HCurl(mesh, order=0, type1=True)

In [ ]:
shapenumber = 15
shape = GridFunction(N, name='shape')
shape.vec[:] = 0
shape.vec[shapenumber] = 1
Draw(shape, mesh, vectors={'grid_size': 30});

As you can see, this shape function visually appears to be exactly the same shape function plotted earlier for the RT case, except for a rotation by 90 degrees.  Note how the tangential component of the shape function varies continuously across the edges. Just as the RT space was useful for interpolating vector fields with normal continuity, the Nedelec space is useful for interpolating vector fields with tangential continuity. 

The Nedelec finite element is sometimes called the **edge finite element** in some engineering literature. Interestingly, the shape functions of the RT and the Nedelec element were known in other mathematical context (they appeared in a 1957 book by Whitney) long before these elements were known. In recognition of this, mathematicians sometimes refer to these shape functions as **Whitney forms**. Here is a formula for the shape functions (of the lowest order Nedelec space) that you just saw in the above visualization:

**Exercise 1.**   Consider an edge $e_{ij}$ between adjacent mesh vertices $a_i , a_j$ of a triangular mesh. Let  $\lambda_i$ denote the hat function (aka barycentric coordinate  function) associated to the vertex $a_i$. Show that the function

$$\newcommand{\grad}{\mathop{\mathrm{grad}}}
    \phi_{ij} = \lambda_j \grad \lambda_i - \lambda_i \grad \lambda_j 
$$

has zero tangential components on all mesh edges except $e_{ij}$,  where it equals $ \phi_{ij} \cdot t_{ij} = 1/ |e_{ij}|$. Here  $|e_{ij}|$ is the length of $e_{ij}$ and  $t_{ij} = (a_i - a_j)/ |e_{ij}|$.

## Range of the 2D curl 

Application of the 2D curl operator to a function in the Nedelec space $N_{h, p}$ results in a function with no continuity restrictions across element interfaces in general. Since the application of curl also reduces the polynomial degree within each element, the result is DG function of degree not more than $p$. Thus, we may view $\curl$ as an operator from the Nedelec space to the DG space, 
$$
\curl : N_{h, p}  \to P_{p}(\oh).
$$
*How much of the DG space is filled with the range of $\curl$?*

To answer this,  we will need to understand the range of the above map. Since $\curl$ may act on smooth function spaces or on finite element spaces like  $N_{h, p}$, it is a good idea to disambiguate the specific domain we consider.  So we will write 
$$
\text{' } \curl: N_{h,p}  \text{ '}
$$
to indicate the $\curl$ when it's viewed as an operator acting (only) on $N_{h, p}$ functions. We now proceed to compute the dimension of its range, i.e., the number 
$$\newcommand{\rank}{\mathop{\mathrm{rank}}}
\rank(\curl: N_{h, p}).
$$
To do so, we will need a representation of the operator $\curl: N_{h,p}$  as a matrix obtained using the basis functions of $N_{h,p}$ and $P_{p-1}(\oh)$. Of course, we will also need a domain and a mesh. Let us punch some holes into a square to make the domain a bit more interesting than just a square and proceed. 

In [ ]:
geo = CSG2d()
R0 = Rectangle(pmin=(-1, -1), pmax=(1, 1))
R1 = Rectangle(pmin=(1/8, 1/8), pmax=(1/2,1/2))
R2 = Rectangle(pmin=(-1/2,-1/2), pmax=(-1/8, -1/8))
domain = R0 - R1 - R2
geo.Add(domain)

mesh = ng.Mesh(geo.GenerateMesh(maxh=1/2))
Draw(mesh);

The Nedelec and DG spaces on this mesh for a given degree $p$ are set as follows.

In [ ]:
p = 0
N = ng.HCurl(mesh, order=p+1-max(1-p,0), type1=True)
D = ng.L2(mesh, order=p)

One way to obtain the matrix of $\curl$ is to take each shape function in $N_{h, p}$, apply curl, and then expand the result in terms of the shape functions of the DG space. However, since the application of `curl` in NGSolve does not output an object in the DG space, we need to do an additional step to make this computationally feasible. One idea is to use the `Set` method of the DG space: if you provide it a function that you know should be contained within the DG space, then the `Set` method interpolates it with zero error, creating the interpolant as a `GridFunction` object. Such objects can be queried for their vector of coefficients of basis expansions, which become entries of the curl matrix. This idea is implemented below.

In [ ]:
from ngsolve import curl 
import numpy as np

def curlmatrix(N, D):
    """Return the matrix representation of curl: N -> D"""
    curlmat = []
    shapeN = GridFunction(N)
    curlD = GridFunction(D)
    for i in range(N.ndof):
        shapeN.vec[:] = 0;  shapeN.vec[i] = 1   # i-th shape function 
        curlD.Set(curl(shapeN))  # can verify that interpolation error is 0
        curlmat.append(list(curlD.vec)) 
    return np.array(curlmat).T

In [ ]:
curlmat = curlmatrix(N, D)

Actually, the code above is grossly inefficient. As you know, python loops are slow. Moreover, inside the above loop, the `Set` does a global computation when what was really needed is just a local computation. NGSolve provides a fast implementation accomplishing the same thing. 

In [ ]:
from ngsolve.comp import ConvertOperator
from scipy.sparse import coo_matrix

u = N.TrialFunction()
Crl = ConvertOperator(N, D, trial_proxy=curl(u))
i, j, val = Crl.COO()
curlmat2 = coo_matrix((val, (i, j))).toarray()

You can verify that `curlmat` and `curlmat2` are exactly the same matrices. Although both methods to arrive at this matrix are viable options for our current simple example, for larger meshes (particularly in 3D), we must remember to perform similar calculations using the faster way and exploit sparse linear algebra for resulting matrix.

Now that we have the matrix representation `curlmat` of the linear operator $\curl: N_{h, p}$, computing its rank can be done by any number of linear algebra techniques. Here is one way, which proceeds by counting the number of nonzero singular values.

In [ ]:
from scipy.linalg import svdvals

s = svdvals(curlmat)
print('Rank of curl =', sum(s>1e-15))

Of course, the rank should be at most the dimension of the codomain, $P_p(\oh)$, which is immediately retrieved from the `ndof` attribute of the finite element space:

In [ ]:
D.ndof

Thus we are computationally led to the interesting discovery, 
$$\newcommand{\rank}{\mathop{\mathrm{rank}}}
\rank(\curl: N_{h, p}) = \dim P_p(\oh),
$$
i.e., <font color=blue>the 2D operator $\curl : N_{h, p} \to P_p(\oh)$ is surjective!</font>  You can certainly hit either of the two simple codes above with other meshes and you will end up with the same observation $\ldots$ so it might be a good idea to add this to the things you should rigorously understand, at least for the lowest order case:

**Exercise 2.** Prove that $\curl: N_{h, 0} \to P_0(\oh)$ is surjective. (You may want to start by computing the curl of the shape functions in Exercise 1.) 

Having discussed the range, let us now briefly turn to the kernel (or the null space) of the same linear operator $\curl:  N_{h, p} \to P_p(\oh)$.   Recalling the rank-nullity theorem from linear algebra, we can immediately compute the dimension of the kernel using the above-computed dimension of the range:
$$
\begin{aligned}
\dim(\ker(\curl:N_{h,p})) 
& = \dim(N_{h, p}) - \rank(\curl:N_{h, p}) \\
& = \dim(N_{h, p}) - \dim P_p(\oh)
\end{aligned}
$$

In [ ]:
dimkercurl = N.ndof - D.ndof
dimkercurl

## The emergence of topology

In calculus, we learned that every curl-free vector field can be written as a gradient of a scalar function, provided the domain has the topological property of being simply connected. However, we punched two holes in our domain, so our domain is not simply connected. Hence we cannot assert that all curl-free functions on our domain are gradients. The number of holes in a two dimensional domain is a topological invariant called the second Betti number $b_1$. (The first Betti number $b_0$ is the number of connected components of the domain, which is clearly 1 for our example.) A deep theorem of de Rham connects the topological  Betti numbers to  quantities arising from smooth function spaces (namely the dimensions of certain cohomology spaces arising from spaces of infinitely smooth functions).  We are now going to see that the number $b_1$ arises very simply and naturally from the above not-so-smooth, but carefully-chosen  spaces of vector finite elements.

Although we do not know that all curl-free functions are gradients on our domain, we do know that within the kernel of $\curl: N_{h, p} \to P_p(\oh)$ there are surely many gradient fields. Since 
$$
\grad v = 
\begin{pmatrix}
\partial_0 v \\ \partial_1 v
\end{pmatrix} 
$$
has vanishing curl, all gradients of functions in the Lagrange space
$$
V_{h, p+1} = \{ v \in H^1(\om):\; \forall K \in \oh, \; v|_K \in P_{p+1}(K)\},
$$
must have zero curl.  Moreover, elementary computation of the gradient and an investigation of the resulting function's tangential component along mesh interfaces proves that 
$$
\grad V_{h, p+1} \subset N_{h, p}.
$$

**Exercise 3:** Prove that $\grad V_{h, p+1} \subset N_{h, p}$.

Thus  the kernel of $\curl: N_{h, p} \to P_p(\oh)$ must contain at least all the gradient fields generated by differentiating Lagrange functions. The size of this gradient subspace, i.e.,  the dimension of  $\grad V_{h, p+1}$,  is easy to compute because we know that *on a connected domain,* the kernel of $\grad: V_{h, p+1}\to N_{h, p}$ consists  of the one-dimensional space of constant functions. Hence, again by the rank-nullity theorem, 
$$
\dim \grad V_{h, p+1} = \dim V_{h, p+1} - 1.
$$

In [ ]:
V = ng.H1(mesh, order=p+1)
dimgradV = V.ndof -1
dimgradV

We computed the exact dimension of the kernel of curl previously. The emergence of topology is evident when we compare the above number with the previously computed $\dim (\ker \curl)$. The number of holes in our domain is obtained as the difference of $\dim(\ker(\curl: N_{h,p}))$ and $\dim(\grad V_{h, p+1})$!

In [ ]:
dimkercurl - dimgradV

Now, I'd like you to go back to the code above, remake the domain with as many holes as you like, change the meshsize to whatever you like, change the polynomial degree $p$, etc., and repeat these computations. (Also see exercises at the end.)

After you have experimented enough, your computational experience should lead you to build some confidence in believing that the second Betti number must be given by 
$$
b_1 = \dim (\ker(\curl: N_{h, p}) - \dim(\grad V_{h, p+1}).
$$
We will have more to say regarding  this in later lectures. For now, we will just note that if you replace the Nedelec space $N_{h, p}$ by the product of Lagrange spaces in the  right hand side above, the resulting number has no topological relevance, as seen below. 

In [ ]:
VV = ng.VectorH1(mesh, order=p+1)

def curl2D(w):
    dw = ng.Grad(w)
    return dw[1, 0] - dw[0, 1]

w = VV.TrialFunction()
curlVV = ConvertOperator(VV, D, trial_cf=curl2D(w))
i, j, val = curlVV.COO()
curlVV = coo_matrix((val, (i, j))).toarray()
rankcurlVV = sum(svdvals(curlVV)>1e-15)
dimkercurlVV = VV.ndof - rankcurlVV
dimkercurlVV - dimgradV

This output is a reflection of the fact that continuity constraints of the product Lagrange space `VV` are too stringent to allow all gradients of $V_{h, p+1}$ to be contained in it. In contrast, the tangential continuity of the Nedelec space is just right to allow $\grad(V_{h, p+1})$ to be contained in it, while at the same time allowing the distributional curl of its vector fields to equal  piecewise polynomials.  

PS: My intent, with this example,  was *not* to present professional methods for computation of Betti numbers of arbitrary domains (a topic in the realm of computational geometry), but rather to invite you into the theoretical area of finite element exterior calculus without relying on too many prerequisites.

# Exercises

**Exercise 1.**   Consider a mesh edge $e_{ij}$ between adjacent mesh vertices $a_i , a_j$ of a triangular mesh. Let  $\lambda_i$ denote the hat function (aka barycentric coordinate  function) associated to the vertex $a_i$. Show that the function

$$
    \phi_{ij} = \lambda_j \grad \lambda_i - \lambda_i \grad \lambda_j 
$$

has zero tangential components on all mesh edges except $e_{ij}$,  where it equals $ \phi_{ij} \cdot t_{ij} = 1/ |e_{ij}|$. Here  $|e_{ij}|$ is the length of $e_{ij}$ and  $t_{ij} = (a_i - a_j)/ |e_{ij}|$.

**Exercise 2.**  Prove that $\curl: N_{h, 0} \to P_0(\oh)$ is surjective on 2D domains.

**Exercise 3.** Prove that $\grad V_{h, p+1} \subset N_{h, p}$.

**Exercise 4.**  Without visualizing a given mesh, how can you tell the number of holes in the domain?  Using NGSolve, write a simple python function that takes as input a mesh and returns $b_1$, the number of holes in the domain. (This is  warm up for the next exercise.)

**Exercise 5.** Somewhere in this notebook we assumed connectedness of the domain. What happens if we remove this assumption? Understanding this, write a python function that takes as input a mesh and  outputs the first Betti number $b_0$, the number of connected components of the domain. (Double-check that your function in  Exercise 4 gives the correct $b_1$ even on domains with $b_0>1,$ revising your answer to Exercise 4, if necessary.)  Display your results from meshes of connected and disconnected domains, with and without holes.

**Exercise 6:** Consider  the  spaces with boundary conditions: 
$$
\begin{aligned}
\mathring{V}_{h, p+1}& = \{ v \in V_{h, p+1}: v|_{\partial \om } = 0\},
\\
\mathring{N}_{h, p}& = \{ u \in N_{h, p}: u\cdot t|_{\partial \om } = 0\}.
\end{aligned}
$$
Determine, computationally or theoretically, if  $\curl : \mathring{N}_{h,0} \to P_0(\oh)$ is surjective. (To do this by NGSolve, use the `FreeDofs` method of finite element spaces.) Is $\grad \mathring{V}_{h, p+1}$ contained in $ \mathring{N}_{h, p}$? Can you  compute $b_0$ and $b_1$ like in Exercise 5 but using $\mathring{V}_{h, p+1}$ and $ \mathring{N}_{h, p}$ instead? 



<hr>
    
    
$\ll$ [Table Of Contents](./INDEX.ipynb)
<br>
$\ll$ [Jay Gopalakrishnan](http://web.pdx.edu/~gjay/)

    

